## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
threshold = 500
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() <= threshold ].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts > 1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
threshold = 1000
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() <= threshold ].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)

In [10]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop('IS_SUCCESSFUL', axis=1)
y = application_df_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Optimization Attempt 1:

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 15

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                1760      
                                                                 
 dense_1 (Dense)             (None, 15)                615       
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 2391 (9.34 KB)
Trainable params: 2391 (9.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
path = "model_weights/weights.{epoch:02d}.hdf5"

# Callback to save model weights every 5 epochs
callback = ModelCheckpoint(
    filepath=path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5759 - accuracy: 0.7130
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7282
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5520 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7302
Epoch 5/100
798/804 [============================>.] - ETA: 0s - loss: 0.5496 - accuracy: 0.7308
Epoch 5: saving model to model_weights/weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5495 - accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5488 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accuracy: 0.7334
Epoch 9/100
804/804 [==============================]

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5521 - accuracy: 0.7311 - 303ms/epoch - 1ms/step
Loss: 0.552147626876831, Accuracy: 0.7310787439346313


### Optimization Attempt 2:

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  120
hidden_nodes_layer2 = 45

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 120)               5280      
                                                                 
 dense_4 (Dense)             (None, 45)                5445      
                                                                 
 dense_5 (Dense)             (None, 1)                 46        
                                                                 
Total params: 10771 (42.07 KB)
Trainable params: 10771 (42.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
path = "model_weights/weights2.{epoch:02d}.hdf5"

# Callback to save model weights every 5 epochs
callback = ModelCheckpoint(
    filepath=path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [15]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback])

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5701 - accuracy: 0.7224
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7279
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5529 - accuracy: 0.7284
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7310
Epoch 5/100
803/804 [============================>.] - ETA: 0s - loss: 0.5499 - accuracy: 0.7313
Epoch 5: saving model to model_weights/weights2.05.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5491 - accuracy: 0.7317
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5481 - accuracy: 0.7339
Epoch 9/100
804/804 [==============================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7273 - 315ms/epoch - 1ms/step
Loss: 0.5595431327819824, Accuracy: 0.7273469567298889


### Optimization Attempt 3:

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train.shape[1]
hidden_nodes_layer1 =  40
hidden_nodes_layer2 = 15

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 40)                1760      
                                                                 
 dense_5 (Dense)             (None, 15)                615       
                                                                 
 dense_6 (Dense)             (None, 15)                240       
                                                                 
 dense_7 (Dense)             (None, 1)                 16        
                                                                 
Total params: 2631 (10.28 KB)
Trainable params: 2631 (10.28 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
path = "model_weights/weights3.{epoch:02d}.hdf5"

# Callback to save model weights every 5 epochs
callback = ModelCheckpoint(
    filepath=path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [15]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100, callbacks=[callback])

Epoch 1/100
804/804 [==============================] - 5s 3ms/step - loss: 0.5827 - accuracy: 0.7103
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7280
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5519 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5503 - accuracy: 0.7312
Epoch 5/100
798/804 [============================>.] - ETA: 0s - loss: 0.5492 - accuracy: 0.7331
Epoch 5: saving model to model_weights/weights3.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5482 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7317
Epoch 9/100
804/804 [==============================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7277 - 325ms/epoch - 1ms/step
Loss: 0.5548498630523682, Accuracy: 0.7276967763900757
